In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
from config import username, password

In [2]:
# DB Connection
hostname = "localhost"  # usually is localhost 
port = "5432" # usually is 5432 
database = "wellwell_db" 
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')
engine.execute("DELETE FROM borehole_observation")
engine.execute("DELETE FROM basin")
engine.execute("DELETE FROM biostrat")
engine.execute("DELETE FROM well_report")
engine.execute("DELETE FROM elevation")
engine.execute("DELETE FROM well_header")
engine.execute("DELETE FROM borehole")
conn = engine.connect()

In [3]:
# Data reference
plover1_xlsx = 'Reports/Plover1.xlsx'
plover1_html = 'html_files/plover_1.html'

plover2_xlsx = 'Reports/Plover1.xlsx'
plover2_html = 'html_files/plover_1.html'

whimbrel1_xlsx = 'Reports/Whimbrel1.xlsx'
whimbrel1_html = 'html_files/Whimbrel_1.html'

nopims = 'Reports/nopims.xlsx'

plover_one = 1
plover_two = 2
whimbrel_one = 3

# NOPIMS Source

In [4]:
# Read data from source and filter required data
dtable = pd.read_excel(nopims)
dtable.sort_values(by=['Well'], inplace=True)
bore_dtable = dtable[dtable["Well"].isin(['Plover 1','Plover 2','Whimbrel 1'])]
bore_dtable

,UWI,Well,UBHI,Borehole,Well aliases,State,Jurisdiction,Offshore,Basin,Sub-basin,...,Oil show,Gas show,Condensate show,Oil field,Discovery well,Well head,Basic relevant date,Interp relevant date,Basic release date,Interp release date
69,ENO0014880,Plover 1,ENO0014880,Plover 1,NaN,WA,Commonwealth,Y,Bonaparte/Northern Bonaparte,Londonderry High/Petrel Sub-basin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
67,ENO0014881,Plover 2,ENO0014881,Plover 2,NaN,WA,Commonwealth,Y,Bonaparte/Northern Bonaparte,Petrel Sub-basin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
68,ENO0016958,Whimbrel 1,ENO0016958,Whimbrel 1,NaN,WA,Commonwealth,Y,Bonaparte,Londonderry High,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT


In [5]:
# Filter required column
borehole_filter=bore_dtable.iloc[:,[1,2,5,8,9,10,11,14,15,22]]
borehole = borehole_filter.rename(columns={'Well':'borehole_name','UBHI':'ubhi','Well operator':'operator','State':'state','Basin':'basin','Sub-basin':'sub_basin','Latitude':'latitude','Longitude':'longitude','Title':'title','Rigs':'rigs'})
borehole.reset_index(drop=True)
borehole.index = np.arange(1, len(borehole) + 1)
borehole_final = borehole.rename_axis('well_id')
nopims_final = borehole_final.dropna()
nopims_final

,borehole_name,ubhi,state,basin,sub_basin,latitude,longitude,title,operator,rigs
well_id,,,,,,,,,,
1,Plover 1,ENO0014880,WA,Bonaparte/Northern Bonaparte,Londonderry High/Petrel Sub-basin,-12.711113,126.369859,WA-16-P,ARCO,Sedco 135G
2,Plover 2,ENO0014881,WA,Bonaparte/Northern Bonaparte,Petrel Sub-basin,-12.956669,126.175695,WA-16-P,ARCO,Margie
3,Whimbrel 1,ENO0016958,WA,Bonaparte,Londonderry High,-12.481395,125.379309,WA-15-P,ARCO,Margie


# Well Header Data

In [6]:
# Filter required column for well_header table
well_header = nopims_final.iloc[:,[2,8,5,6,9,1]]
well_header

,state,operator,latitude,longitude,rigs,ubhi
well_id,,,,,,
1,WA,ARCO,-12.711113,126.369859,Sedco 135G,ENO0014880
2,WA,ARCO,-12.956669,126.175695,Margie,ENO0014881
3,WA,ARCO,-12.481395,125.379309,Margie,ENO0016958


In [7]:
# Insert data to well_header data
well_header.to_sql(name='well_header', con=engine, if_exists='append', index=True)

# Basin Data

In [8]:
# Filter required column for basin table
basin1 = nopims_final.iloc[:,[3,4]]
basin = basin1.rename_axis('basin_id')
basin

,basin,sub_basin
basin_id,,
1,Bonaparte/Northern Bonaparte,Londonderry High/Petrel Sub-basin
2,Bonaparte/Northern Bonaparte,Petrel Sub-basin
3,Bonaparte,Londonderry High


In [9]:
# Insert data to basin table
basin.to_sql(name='basin', con=engine, if_exists='append', index=True)

# Borehole Data

In [10]:
# Filter required column for borehole table
borehole1 = nopims_final.iloc[:,[0]]
borehole2 = borehole1.rename(columns={'Borehole':'borehole_name'})
borehole = borehole2.rename_axis('borehole_id')
borehole

,borehole_name
borehole_id,
1,Plover 1
2,Plover 2
3,Whimbrel 1


In [11]:
# Insert data to borehole table
borehole.to_sql(name='borehole', con=engine, if_exists='append', index=True)

# Plover1 Well Report Data

In [12]:
# Read from excel source and select required data
dtable = pd.read_excel(plover1_xlsx)
dtable.sort_values(by=['Title'], inplace=True)
reports_dtable = dtable.loc[dtable['Well'] == 'Plover 1']
reports=reports_dtable.iloc[:,[4,5]]

# Rename columns
reports=reports.rename(columns = {'Title':'document_title','Type':'document_type'})

# Reset and rename index, dropna
reports.reset_index(drop=True)
reports.index = np.arange(1, len(reports) + 1)
reports_final = reports.rename_axis('doc_id')
plover1_wellreport = reports_final.dropna()
plover1_wellreport['borehole_id'] = plover_one
plover1_wellreport

,document_title,document_type,borehole_id
doc_id,,,
1,Destructive Analysis;A Palynological Study of ...,Destructive Analysis,1
2,Destructive Analysis;A Palynologoical Study of...,Destructive Analysis,1
3,Destructive Analysis;Apatite Fission Track and...,Destructive Analysis,1
4,Destructive Analysis;Bulk Fluid Inclusion Mass...,Destructive Analysis,1
5,Destructive Analysis;Geochem study of samples ...,Destructive Analysis,1
6,Destructive Analysis;Geochemical evaluation re...,Destructive Analysis,1
7,Destructive Analysis;Hydrocarbon Petrography R...,Destructive Analysis,1
8,Destructive Analysis;New Palynology Data from ...,Destructive Analysis,1
9,Destructive Analysis;New Palynology from Plove...,Destructive Analysis,1


# Plover2 Well Report Data

In [13]:
# Read from excel source and select required data
dtable = pd.read_excel(plover2_xlsx)
dtable.sort_values(by=['Title'], inplace=True)
reports_dtable = dtable.loc[dtable['Well'] == 'Plover 1']
reports=reports_dtable.iloc[:,[4,5]]

# Rename columns
reports=reports.rename(columns = {'Title':'document_title','Type':'document_type'})

# Reset and rename index, dropna
reports.reset_index(drop=True)
reports.index = np.arange(1, len(reports) + 1)
reports_final = reports.rename_axis('doc_id')
plover2_wellreport = reports_final.dropna()
plover2_wellreport['borehole_id'] = plover_two
plover2_wellreport

,document_title,document_type,borehole_id
doc_id,,,
1,Destructive Analysis;A Palynological Study of ...,Destructive Analysis,2
2,Destructive Analysis;A Palynologoical Study of...,Destructive Analysis,2
3,Destructive Analysis;Apatite Fission Track and...,Destructive Analysis,2
4,Destructive Analysis;Bulk Fluid Inclusion Mass...,Destructive Analysis,2
5,Destructive Analysis;Geochem study of samples ...,Destructive Analysis,2
6,Destructive Analysis;Geochemical evaluation re...,Destructive Analysis,2
7,Destructive Analysis;Hydrocarbon Petrography R...,Destructive Analysis,2
8,Destructive Analysis;New Palynology Data from ...,Destructive Analysis,2
9,Destructive Analysis;New Palynology from Plove...,Destructive Analysis,2


# Whimbrel1 Well Report Data

In [14]:
# Read from excel source and select required data
dtable = pd.read_excel(whimbrel1_xlsx)
dtable.sort_values(by=['Title'], inplace=True)
reports_dtable = dtable.loc[dtable['Well'] == 'Whimbrel 1']
reports=reports_dtable.iloc[:,[4,5]]

# Rename columns
reports=reports.rename(columns = {'Title':'document_title','Type':'document_type'})

# Reset and rename index, dropna
reports.reset_index(drop=True)
reports.index = np.arange(1, len(reports) + 1)
reports_final = reports.rename_axis('doc_id')
whimbrel1_wellreport = reports_final.dropna()
whimbrel1_wellreport['borehole_id'] = whimbrel_one
whimbrel1_wellreport

,document_title,document_type,borehole_id
doc_id,,,
1,Destructive Analysis;A Palynological Study of ...,Destructive Analysis,3
2,Destructive Analysis;Apatite Fission Track and...,Destructive Analysis,3
3,Destructive Analysis;Biomarker Geochemistry on...,Destructive Analysis,3
4,Destructive Analysis;Biomarker geochemistry on...,Destructive Analysis,3
5,Destructive Analysis;Chemostratigraphy and Dis...,Destructive Analysis,3
6,Destructive Analysis;Geochem study of samples ...,Destructive Analysis,3
7,Destructive Analysis;Hydrocarbon Petrography R...,Destructive Analysis,3
8,Destructive Analysis;Micropalaeontological Ana...,Destructive Analysis,3
9,Destructive Analysis;Northern Browse Basin and...,Destructive Analysis,3


In [15]:
# Read from excel source and select required data
dtable = pd.read_excel(whimbrel1_xlsx)
dtable.sort_values(by=['Title'], inplace=True)
reports_dtable = dtable.loc[dtable['Well'] == 'Whimbrel 1']
reports=reports_dtable.iloc[:,[4,5]]

# Rename columns
reports=reports.rename(columns = {'Title':'document_title','Type':'document_type'})

# Reset and rename index, dropna
reports.reset_index(drop=True)
reports.index = np.arange(1, len(reports) + 1)
reports_final = reports.rename_axis('doc_id')
whimbrel1_wellreport = reports_final.dropna()
whimbrel1_wellreport['borehole_id'] = whimbrel_one
whimbrel1_wellreport

,document_title,document_type,borehole_id
doc_id,,,
1,Destructive Analysis;A Palynological Study of ...,Destructive Analysis,3
2,Destructive Analysis;Apatite Fission Track and...,Destructive Analysis,3
3,Destructive Analysis;Biomarker Geochemistry on...,Destructive Analysis,3
4,Destructive Analysis;Biomarker geochemistry on...,Destructive Analysis,3
5,Destructive Analysis;Chemostratigraphy and Dis...,Destructive Analysis,3
6,Destructive Analysis;Geochem study of samples ...,Destructive Analysis,3
7,Destructive Analysis;Hydrocarbon Petrography R...,Destructive Analysis,3
8,Destructive Analysis;Micropalaeontological Ana...,Destructive Analysis,3
9,Destructive Analysis;Northern Browse Basin and...,Destructive Analysis,3


In [16]:
# Combine well report data to insert to db
frames1 = [plover1_wellreport, plover2_wellreport,whimbrel1_wellreport]
result=pd.concat(frames1)
wellreport_combined=result.reset_index(drop=True)
wellreport_combined.index = np.arange(1, len(wellreport_combined) + 1)
wellreport_combined=wellreport_combined.rename_axis('doc_id')
wellreport_combined

,document_title,document_type,borehole_id
doc_id,,,
1,Destructive Analysis;A Palynological Study of ...,Destructive Analysis,1
2,Destructive Analysis;A Palynologoical Study of...,Destructive Analysis,1
3,Destructive Analysis;Apatite Fission Track and...,Destructive Analysis,1
4,Destructive Analysis;Bulk Fluid Inclusion Mass...,Destructive Analysis,1
5,Destructive Analysis;Geochem study of samples ...,Destructive Analysis,1
...,...,...,...
123,Well log;HDT. Tape image only is stored on the...,Well log,3
124,Well log;HDT.E. Tape image only is stored on t...,Well log,3
125,Well log;Log run 1. ED 606. Tape image only is...,Well log,3


In [17]:
# Insert to db
wellreport_combined.to_sql(name='well_report', con=engine, if_exists='append', index=True)

# Elevation Data

In [18]:
# Read from excel source and select required data
dtable = pd.read_html(plover1_html)
elevation1 = dtable[5].iloc[:,[3]]
elevation_filter = elevation1.iloc[[6,9]]
elevation_filter=elevation_filter.T

# Rename columns
elevation_filter=elevation_filter.rename(columns = {6:'sea_floor', 9:'rotary_table'}, inplace=False)

# Reset and rename index, dropna
elevation_final=elevation_filter.reset_index(drop=True)
elevation_final.index = np.arange(1, len(elevation_final) + 1)
elevation_final= elevation_final.rename_axis('elevation_id')
elevation_one = elevation_final.dropna()
elevation_one

,sea_floor,rotary_table
elevation_id,,
1,-58m MSL,kelly bushing 34m


In [19]:
# Read from excel source and select required data
dtable = pd.read_html(plover2_html)
elevation1 = dtable[5].iloc[:,[3]]
elevation_filter = elevation1.iloc[[6,9]]
elevation_filter=elevation_filter.T

# Rename columns
elevation_filter=elevation_filter.rename(columns = {6:'sea_floor', 9:'rotary_table'}, inplace=False)

# Reset and rename index, dropna
elevation_final=elevation_filter.reset_index(drop=True)
elevation_final.index = np.arange(1, len(elevation_final) + 1)
elevation_final= elevation_final.rename_axis('elevation_id')
elevation_two = elevation_final.dropna()
elevation_two

,sea_floor,rotary_table
elevation_id,,
1,-58m MSL,kelly bushing 34m


In [20]:
# Read from excel source and select required data
dtable = pd.read_html(whimbrel1_html)
elevation1 = dtable[3].iloc[:,[3]]
elevation_filter = elevation1.iloc[[6,9]]
elevation_filter=elevation_filter.T

# Rename columns
elevation_filter=elevation_filter.rename(columns = {6:'sea_floor', 9:'rotary_table'}, inplace=False)

# Reset and rename index, dropna
elevation_final=elevation_filter.reset_index(drop=True)
elevation_final.index = np.arange(1, len(elevation_final) + 1)
elevation_final= elevation_final.rename_axis('elevation_id')
elevation_three = elevation_final.dropna()
elevation_three

,sea_floor,rotary_table
elevation_id,,
1,-76.8m MSL,kelly bushing 83m


In [21]:
# Combine evaluation data to insert to db
frames2 = [elevation_one, elevation_two, elevation_three]
result=pd.concat(frames2)
elevation_combine = result.reset_index(drop=True)
elevation_combine.index = np.arange(1, len(elevation_combine) + 1)
elevation_combine=elevation_combine.rename_axis('elevation_id')
elevation_combine

,sea_floor,rotary_table
elevation_id,,
1,-58m MSL,kelly bushing 34m
2,-58m MSL,kelly bushing 34m
3,-76.8m MSL,kelly bushing 83m


In [22]:
# Insert to evaluation table
elevation_combine.to_sql(name='elevation', con=engine, if_exists='append', index=True)

# Biostrat Data

In [23]:
# Read from excel source and select required data
dtable = pd.read_html(plover1_html)
biostrat_data1 = dtable[-2]
biostrat_filter1 = biostrat_data1.iloc[:,[0,1]]
biostrat_filter2 = biostrat_filter1.iloc[4:]

# Rename columns
biostrat_filter2 = biostrat_filter2.rename(columns = {0:'depth', 1:'zone',4:'class', 5:'readings',6:'age'}, inplace=False)

# Reset and rename index, dropna
biostrat_filter2.reset_index(drop=True)
biostrat_filter2.index = np.arange(1, len(biostrat_filter2) + 1)
biostrat_final= biostrat_filter2.rename_axis('biostrat_id')
biostrat1 = biostrat_final.dropna()
biostrat1['borehole_id'] = plover_one
biostrat1

C:\Users\rayca\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,depth,zone,borehole_id
biostrat_id,,,
1,396,Diconodinium multispinum 1aii,1
2,396,Appendicisporites distocarinatus,1
3,809,Pseudoceratium ludbrookiae 1b,1
4,992,Muderongia australis 2diii Subzone,1
5,1027,Muderongia australis 2di-2diii,1
6,1060,Egmontodinium torynum 4ai-4aii,1
7,1085,Dissimulidinium lobispinosum 4bi,1
8,1091,Kalyptea wisemaniae 4biii,1
9,1097,Dingodinium jurassicum 5a-b,1


In [24]:
# Read from excel source and select required data
dtable = pd.read_html(plover2_html)
biostrat_data2 = dtable[-2]
biostrat_filter1 = biostrat_data2.iloc[:,[0,1]]
biostrat_filter2 = biostrat_filter1.iloc[4:]

# Rename columns
biostrat_filter2 = biostrat_filter2.rename(columns = {0:'depth', 1:'zone',4:'class', 5:'readings',6:'age'}, inplace=False)

# Reset and rename index, dropna
biostrat_filter2.reset_index(drop=True)
biostrat_filter2.index = np.arange(1, len(biostrat_filter2) + 1)
biostrat_final= biostrat_filter2.rename_axis('biostrat_id')
biostrat2 = biostrat_final.dropna()
biostrat2['borehole_id'] = plover_two
biostrat2

C:\Users\rayca\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,depth,zone,borehole_id
biostrat_id,,,
1,396,Diconodinium multispinum 1aii,2
2,396,Appendicisporites distocarinatus,2
3,809,Pseudoceratium ludbrookiae 1b,2
4,992,Muderongia australis 2diii Subzone,2
5,1027,Muderongia australis 2di-2diii,2
6,1060,Egmontodinium torynum 4ai-4aii,2
7,1085,Dissimulidinium lobispinosum 4bi,2
8,1091,Kalyptea wisemaniae 4biii,2
9,1097,Dingodinium jurassicum 5a-b,2


In [25]:
# Read from excel source and select required data
dtable = pd.read_html(whimbrel1_html)
biostrat_data2 = dtable[-3]
biostrat_filter1 = biostrat_data2.iloc[:,[0,1]]
biostrat_filter2 = biostrat_filter1.iloc[4:]

# Rename columns
biostrat_filter2 = biostrat_filter2.rename(columns = {0:'depth', 1:'zone',4:'class', 5:'readings',6:'age'}, inplace=False)

# Reset and rename index, dropna
biostrat_filter2.reset_index(drop=True)
biostrat_filter2.index = np.arange(1, len(biostrat_filter2) + 1)
biostrat_final= biostrat_filter2.rename_axis('biostrat_id')
biostrat3 = biostrat_final.dropna()
biostrat3['borehole_id'] = whimbrel_one
biostrat3

C:\Users\rayca\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,depth,zone,borehole_id
biostrat_id,,,
1,237.13,Pliocene,3
2,368.8,Tf1,3
3,368.8,Te1-4,3
4,647.7,Isabelidinium Superzone,3
5,647.7,Globotruncana aegyptica Zone,3
6,647.7,Dicarinella asymetrica Zone,3
7,659.9,Isabelidinium Superzone,3
8,675.13,Helvetoglobotruncana helvetica Zone,3
9,757.4,Diconodinium multispinum 1aii,3


In [26]:
# Combine biostrat data to insert to db
frames3 = [biostrat1, biostrat2,biostrat3]
biostrat_combined=pd.concat(frames3)
biostrat_combined.reset_index(drop=True)
biostrat_combined.index = np.arange(1, len(biostrat_combined) + 1)
biostrat_combined=biostrat_combined.rename_axis('biostrat_id')
biostrat_combined

,depth,zone,borehole_id
biostrat_id,,,
1,396,Diconodinium multispinum 1aii,1
2,396,Appendicisporites distocarinatus,1
3,809,Pseudoceratium ludbrookiae 1b,1
4,992,Muderongia australis 2diii Subzone,1
5,1027,Muderongia australis 2di-2diii,1
...,...,...,...
76,1878.8,Lunatisporites pellucidus,3
77,2001.6,Lunatisporites pellucidus,3
78,2043.7,STAGE 5,3


In [27]:
# Insert to biostrat table
biostrat_combined.to_sql(name='biostrat', con=engine, if_exists='append', index=True)

# Biostrat_Observation Data

In [28]:
# Set dataframe columns
bio_obs = pd.DataFrame(columns = ['observation_id', 'borhole_id','basin_id','doc_id','biostrat_id','elevation_id','well_id'])

# Values array
columns = list(bio_obs)
data = []

# for loop to get data for borehole_observation
for i in elevation_combine.index:
  values = [i, i, i, i, i, i, i]
  zipped = zip(columns, values)
  a_dictionary = dict(zipped)
  print(a_dictionary)
  data.append(a_dictionary)
bio_obs = bio_obs.append(data, True)
bio_obs

{'observation_id': 1, 'borhole_id': 1, 'basin_id': 1, 'doc_id': 1, 'biostrat_id': 1, 'elevation_id': 1, 'well_id': 1}
{'observation_id': 2, 'borhole_id': 2, 'basin_id': 2, 'doc_id': 2, 'biostrat_id': 2, 'elevation_id': 2, 'well_id': 2}
{'observation_id': 3, 'borhole_id': 3, 'basin_id': 3, 'doc_id': 3, 'biostrat_id': 3, 'elevation_id': 3, 'well_id': 3}


,observation_id,borhole_id,basin_id,doc_id,biostrat_id,elevation_id,well_id
0,1,1,1,1,1,1,1
1,2,2,2,2,2,2,2
2,3,3,3,3,3,3,3


In [29]:
# Insert to borehole_observation table
bio_obs.to_sql(name='borehole_observation', con=engine, if_exists='append', index=False)